## Toronto Cruster Analysis

#scrape toronto data from wiki

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))

In [2]:
df = df[0]

In [3]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#Cleanning 'Not Assigned' Data

In [4]:
newdf = df

In [5]:
newdf = newdf[newdf.Borough !='Not assigned']

In [6]:
newdf.Neighbourhood.size

212

In [7]:
for i in range(newdf.Neighbourhood.size):
    if newdf['Neighbourhood'].values[i] == 'Not assigned':
        newdf['Neighbourhood'].values[i] = newdf['Borough'].values[i]

In [8]:
newdf.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


#group Postcode and join Neighbourhood

In [9]:
Mdf = newdf.groupby(['Postcode','Borough'])['Neighbourhood'].agg(', '.join).reset_index()

In [10]:
Mdf.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
Mdf.shape

(103, 3)

#Read Toronto Geo CSV file

In [12]:
rawdf = pd.read_csv('http://cocl.us/Geospatial_data')
rawdf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#Merge data frame

In [13]:
Mdf['Latitude'] = Mdf.Postcode.map(rawdf.set_index('Postal Code')['Latitude'].to_dict())
Mdf['Longitude'] = Mdf.Postcode.map(rawdf.set_index('Postal Code')['Longitude'].to_dict())
Mdf.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
